### Import Moduels

In [1]:
%matplotlib inline
from IPython.display import display

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib import rcParams
import seaborn as sns

import requests                   
import json

import datetime as dt
from time import time

from IPython.display import display

### Obtain All Names of Perpetuals

In [2]:
url = 'https://ftx.com/api/futures'

futures_raw = pd.DataFrame(json.loads(requests.get(url).text))

index = futures_raw.iloc[0,1].keys()
futures_sorted = pd.DataFrame(index=index)

for row in range(len(futures_raw)):
    future_data = pd.DataFrame(futures_raw.iloc[row,1].values(),index=index)
    futures_sorted[str(row)] = future_data

columns = futures_sorted.iloc[0].tolist()
futures_sorted.columns = columns
futures_sorted

perpetuals = pd.DataFrame(index=index)
for column in futures_sorted.columns.tolist():
    if 'perpetual' in futures_sorted[column].tolist():
        perpetuals[column] = futures_sorted[column].tolist()
        
perpetuals = perpetuals.transpose()
perpeturals_list = perpetuals.index.tolist()

### Order Book Liquidity

In [3]:
perpetual_L = []  
size_change_L = [] 
ask_price_raw_L = []
size_change_ask_price_raw_L = []

for perpetual in perpeturals_list:
    
    # Define parameters
    perpetual = perpetual
    depth = '100'

    # Extract Data
    url = 'https://ftx.com/api/markets/'+perpetual+'/orderbook?depth='+depth
    orderbook_raw = pd.DataFrame(json.loads(requests.get(url).text))
    asks_raw = orderbook_raw.iloc[0,1]
    bids_raw = orderbook_raw.iloc[1,1]
    orderbook_raw

    ask_price = []
    ask_size = []
    bid_price = []
    bid_size = []

    for ask,bid in zip(asks_raw,bids_raw):

        ask_price_raw = ask[0]
        ask_size_raw = ask[1]
        ask_price.append(ask_price_raw)
        ask_size.append(ask_size_raw)
        bid_price_raw = bid[0]
        bid_size_raw = bid[1]
        bid_price.append(bid_price_raw)
        bid_size.append(bid_size_raw)

    orderbook_sorted = pd.DataFrame()
    orderbook_sorted['Ask Price'] = ask_price
    orderbook_sorted['Ask Size'] = ask_size
    orderbook_sorted['Cum. Ask Size'] = orderbook_sorted['Ask Size'].cumsum()
    orderbook_sorted['Bid Price'] = bid_price
    orderbook_sorted['Bid Size'] = bid_size
    orderbook_sorted['Cum. Bid Size'] = orderbook_sorted['Bid Size'].cumsum()
    orderbook_sorted.index.name = perpetual
    orderbook_sorted

    percent = 1
    percent =percent/100
    try:
        first_price = ask_price[0]
    except IndexError:
        continue
    target_price = first_price*(percent+1)
    last_price = ask_price[-1]

    if target_price in ask_price:
        index = ask_price.index(target_price)
        target_price = ask_price[index]
        size_change = orderbook_sorted.iloc[index,2]

    elif last_price>target_price:
        differences = []
        for price in ask_price:
            difference = abs(target_price-price)
            differences.append(difference)
        smallest_difference = min(differences)
        index = differences.index(smallest_difference)
        target_price = ask_price[index]
        size_change = orderbook_sorted.iloc[index,2]

    else:
        size_change = orderbook_sorted.iloc[-1,2]

    percent = percent*100
    
    perpetual_L.append(perpetual)
    size_change_L.append(size_change)
    ask_price_raw_L.append(ask_price_raw)
    size_change_ask_price_raw_L.append(size_change*ask_price_raw)
    
liquidity_df = pd.DataFrame()
liquidity_df['Coin'] = perpetual_L
liquidity_df['Size'] = size_change_L
liquidity_df['Ask'] = ask_price_raw_L
liquidity_df['USD needed to increase price by'+str(percent)+'%'] = size_change_ask_price_raw_L
liquidity_df = liquidity_df.sort_values(by=['USD needed to increase price by'+str(percent)+'%'])
most_liquid = liquidity_df.tail(20)
least_liquid = liquidity_df.head(20)

In [4]:
liquidity_df

,Coin,Size,Ask,USD needed to increase price by1.0%
128,SHIB-PERP,-9.990000e+07,0.000048,-4.831164e+03
136,SRN-PERP,1.177400e+05,0.014750,1.736665e+03
122,ROOK-PERP,2.471100e+01,185.150000,4.575242e+03
93,MCB-PERP,1.980600e+02,33.920000,6.718195e+03
9,AMPL-PERP,1.126000e+04,0.927600,1.044478e+04
...,...,...,...,...
61,EXCH-PERP,2.538200e+02,20000.500000,5.076527e+06
60,ETH-PERP,2.074592e+03,4393.700000,9.115135e+06
8,ALT-PERP,1.580938e+03,5963.000000,9.427133e+06
29,BTC-PERP,2.101710e+02,59504.000000,1.250602e+07


In [5]:
most_liquid

,Coin,Size,Ask,USD needed to increase price by1.0%
63,FIL-PERP,2.695980e+04,54.895000,1.479958e+06
55,EGLD-PERP,4.389860e+03,364.640000,1.600719e+06
92,MATIC-PERP,1.007889e+06,1.632825,1.645706e+06
158,XLM-PERP,4.808724e+06,0.350700,1.686420e+06
151,UNI-PERP,8.216710e+04,21.879000,1.797734e+06
5,ALGO-PERP,9.837570e+05,1.840200,1.810310e+06
164,ZEC-PERP,1.053785e+04,182.420000,1.922315e+06
133,SOL-PERP,9.058870e+03,215.647500,1.953523e+06
102,MVDA25-PERP,4.445860e+01,50000.000000,2.222930e+06
101,MVDA10-PERP,4.214040e+01,54000.000000,2.275582e+06


In [6]:
least_liquid

,Coin,Size,Ask,USD needed to increase price by1.0%
128,SHIB-PERP,-9.990000e+07,0.000048,-4831.16400
136,SRN-PERP,1.177400e+05,0.014750,1736.66500
122,ROOK-PERP,2.471100e+01,185.150000,4575.24165
93,MCB-PERP,1.980600e+02,33.920000,6718.19520
9,AMPL-PERP,1.126000e+04,0.927600,10444.77600
95,MER-PERP,2.491900e+04,0.510800,12728.62520
94,MEDIA-PERP,2.122100e+02,83.600000,17740.75600
149,TRYB-PERP,1.963720e+05,0.092610,18186.01092
33,CEL-PERP,4.265200e+03,4.498000,19184.86960
99,MTA-PERP,1.585500e+04,1.275500,20223.05250
